# Collecting weather data from an API

In [1]:
import requests
import pandas as pd
import numpy as np
from keys import *

city = "Berlin"
country = "DE"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [17]:
data = response.json()

# Extract forecast list
forecast_list = data.get('list', [])

# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Berlin,DE"
})

print(df.head())

      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-29 21:00:00         9.61        87          Clear  4.14       NaN   
1  2025-10-30 00:00:00         9.51        83         Clouds  4.46       NaN   
2  2025-10-30 03:00:00         9.84        83           Rain  4.10      0.37   
3  2025-10-30 06:00:00        10.94        80           Rain  7.48      2.95   
4  2025-10-30 09:00:00        12.28        83           Rain  9.01      3.41   

   snow municipality_iso_country  
0   NaN                Berlin,DE  
1   NaN                Berlin,DE  
2   NaN                Berlin,DE  
3   NaN                Berlin,DE  
4   NaN                Berlin,DE  


In [18]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Vimaro77"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40